## Создание текстовых файлов на основе pdf

In [10]:
import os
import PyPDF2
import pytesseract
from pdf2image import convert_from_path

import glob
import tqdm
import re

In [13]:
pdfs = glob.glob('/home/yagor/Рабочий стол/atomic/реальныепдф/*.pdf')

In [14]:
len(pdfs)

30

In [ ]:
input_dir = '/home/yagor/Рабочий стол/atomic/реальныепдф'
output_dir = '/home/yagor/Рабочий стол/atomic/AtomHack/data/txt_data'
file_names = os.listdir(input_dir)

for file_name in tqdm.tqdm([f for f in file_names if f.endswith(".pdf")]):
    pagewise_text_list = []
    pdf_document = os.path.join(input_dir, file_name)
    with open(pdf_document, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(reader.pages)

        # Iterate through the pages and extract text
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text = page.extract_text() # TODO: what happens if there is a table?

            if len(re.sub('[^A-Za-z0-9]+', '', text)) == 0: # let's use OCR
                images = convert_from_path(pdf_document, first_page=page_num + 1, last_page=page_num + 1)
                text = ''
                for image_num in range(len(images)):
                    text += pytesseract.image_to_string(images[image_num], lang='rus')

            pagewise_text_list.append(text)
            output_file_path = os.path.join(output_dir, file_name.replace(".pdf", ""))
            if not os.path.exists(output_file_path):
                os.makedirs(output_file_path)

            for page_num, text in enumerate(pagewise_text_list):
                page_file_path = os.path.join(output_file_path, f"page_{page_num + 1}.txt")
                with open(page_file_path, "w") as page_file:
                    page_file.write(text)